# Catalogue Disambiguation: `search` -> `disambiguate` -> `filter`

Continues from `2020_06_26 searching items by name.ipynb`.

In [1]:
import sys
sys.path.append("..")

from heritageconnector.utils.sparql import get_sparql_results

import re
from tqdm import tqdm
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

tqdm.pandas()

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## 1. Load data and ID mapping

In [2]:
id_mapping = pd.read_csv("../GITIGNORE_DATA/itemnames_links.csv")

# get rid of IDs for which we don't have a qcode
id_mapping = id_mapping[id_mapping["wikidata_id"].astype(str).str.startswith("Q")]
names_mapped = id_mapping['item_name'].tolist()

id_mapping.head(2)

,item_name,count,match_name,match_id,getty_aat_id,wikidata_id
0,photograph,11087.0,photographs,aat/300046300,300046300.0,Q125191
1,poster,10003.0,posters,aat/300027221,300027221.0,Q429785


In [3]:
df = pd.read_csv("../GITIGNORE_DATA/smg-datasets-private/mimsy-catalogue-export.csv")

# clean up title column so it's searchable in the SPARQL query
df["TITLE"] = df["TITLE"].str.replace("\"", "").replace("\n", "")

# make list from ITEM_NAME column
df["ITEM_NAME_list"] = df['ITEM_NAME'].fillna("").astype(str).apply(lambda i: [x.strip().lower() for x in i.split(';')])

# filter df to only items which have properties we have matched
df['name_mapped'] = df['ITEM_NAME_list'].progress_apply(lambda x: any([item in names_mapped for item in x]) )
dfm = df[df['name_mapped'] == True]

# percentage, number of items for which we have a property that can be mapped:
print(len(dfm) / len(df) * 100, len(dfm))

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,6,8,12,18,25,26,27,30,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 282259/282259 [00:06<00:00, 44662.71it/s]


33.069627540663014 93342


## 2. Get qcodes from item names using mapping table

In [4]:
def qcode_from_name(name):
    try:
        return id_mapping.loc[id_mapping["item_name"] == name, 'wikidata_id'].values[0]
    except:
        return None

dfm['ITEM_NAME_qcodes'] = dfm['ITEM_NAME_list'].apply(lambda l: [qcode_from_name(i) for i in l if qcode_from_name(i) not in (None, )])

dfm['ITEM_NAME_qcodes'].apply(len).value_counts()

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


1    85357
2     7466
3      476
4       40
5        2
7        1
Name: ITEM_NAME_qcodes, dtype: int64

## 3. `search`: fine-tuning SPARQL query
Item properties used for query:
- name (text search)
- item_name (Wikidata qcode search)

### 3.1 Filter stopwords
Any item names that are 'generic'.

In [5]:
itemnames_unique = list(set(df['ITEM_NAME_list'].sum()))
itemnames_unique = [item for item in itemnames_unique if item not in [""]]

stopwords = itemnames_unique
stopwords += ['statue of buddha', 'pin', 'pencil drawing', 'hindu astrolabe', 'glass bottle'] # custom stopwords

title_count = dfm['TITLE'].astype(str).str.lower().value_counts()
unique_titles = title_count.index.tolist()
print("short/long...")
stopwords += [title for title in unique_titles if len(title)<= 3 or len(title)>=50] # short/long strings
print("frequent...")
stopwords += [title for title in unique_titles if title_count[title] > 1] # frequent strings

print('performing filtering...')
len_before = len(dfm)
dfm = dfm[~dfm['TITLE'].astype(str).str.lower().isin(stopwords)]

len(dfm)/len_before, len(dfm)

short/long...
frequent...
performing filtering...


(0.3803539671316235, 35503)

### 3.2 Run search

In [6]:
endpoint_url = "https://query.wikidata.org/sparql"
    
def run_query(mkey, title, itemnames):
    # NOTE: the LIMIT value at the end also tells the query how far up to go in the 'class of' chain
    map_ids = lambda ids: ",".join([f"wd:{i}" for i in ids])
    
    query = f"""
    SELECT DISTINCT ?item ?itemLabel ?type ?typeLabel WHERE {{
      SERVICE wikibase:mwapi {{
          bd:serviceParam wikibase:api "EntitySearch" .
          bd:serviceParam wikibase:endpoint "www.wikidata.org" .
          bd:serviceParam mwapi:search "{title}" .
          bd:serviceParam mwapi:language "en" .
          ?item wikibase:apiOutputItem mwapi:item .
          ?num wikibase:apiOrdinal true .
      }}
      ?item (wdt:P279|wdt:P31) ?type .
      ?item (wdt:P31/wdt:P279*) ?classTree .
      FILTER ( ?classTree in ( {map_ids(itemnames)} ) )
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en"}}
    }} ORDER BY ASC(?num) LIMIT 20
    """
    
    res = get_sparql_results(endpoint_url, query)['results']['bindings']
    res_df = pd.json_normalize(res)
    if len(res_df) > 0:
        res_df["MKEY"] = mkey
        res_df["TITLE"] = title
        res_df = res_df[['MKEY', 'TITLE', 'item.value', 'itemLabel.value', 'type.value', 'typeLabel.value']]
        res_df = res_df.drop_duplicates(subset=["MKEY", "TITLE", "item.value", "itemLabel.value"])
    return res_df


In [8]:
query_res = pd.DataFrame()

for idx, row in tqdm(dfm.iterrows(), total=dfm.shape[0]):
    mkey = row["MKEY"]
    title = row["TITLE"]
    itemnames = row["ITEM_NAME_qcodes"]
    
    try:
        tempdf = run_query(mkey, title, itemnames)
        query_res = pd.concat([query_res, tempdf])
    
        if len(tempdf) > 0: 
            print(f"{mkey}, {title}: {len(tempdf)}")
    except:
        print(f"{mkey}, {title} FAILED")

  0%|          | 173/35503 [01:01<3:53:06,  2.53it/s]

1347, Wells Cathedral clock: 1


  1%|▏         | 450/35503 [02:53<4:01:50,  2.42it/s]

9367, Bleach Works at Llewenni: 1


  2%|▏         | 689/35503 [04:24<6:21:00,  1.52it/s] 

...

 99%|█████████▉| 35303/35503 [3:26:17<00:59,  3.34it/s]

8653407, Notre-Dame: 1


 99%|█████████▉| 35316/35503 [3:26:22<01:08,  2.73it/s]

8670122, Mending nets: 1


100%|██████████| 35503/35503 [3:27:18<00:00,  2.85it/s]


In [12]:
query_res.to_csv('../GITIGNORE_DATA/disambiguation/objects_lookup_1.csv', index=False)
query_res.head()

,MKEY,TITLE,item.value,itemLabel.value,type.value,typeLabel.value
0,1347,Wells Cathedral clock,http://www.wikidata.org/entity/Q11156104,Wells Cathedral clock,http://www.wikidata.org/entity/Q5275,astronomical clock
0,9367,Bleach Works at Llewenni,http://www.wikidata.org/entity/Q23832039,"Bleach works at Llewenni: as at first intended to be built for the honble Thos Fitzmaurice, Denbighshire",http://www.wikidata.org/entity/Q11060274,print
0,12627,Paracelsus,http://www.wikidata.org/entity/Q55014956,Paracelsus,http://www.wikidata.org/entity/Q11060274,print
0,27748,Leopard,http://www.wikidata.org/entity/Q83553778,Leopard,http://www.wikidata.org/entity/Q3305213,painting
1,27748,Leopard,http://www.wikidata.org/entity/Q19883049,Leopard,http://www.wikidata.org/entity/Q3305213,painting
